In [2]:
# === 1. IMPORTS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import joblib
import optuna

# Scikit-learn modules
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error
from sklearn.base import BaseEstimator, RegressorMixin

# XGBoost Regressor
from xgboost import XGBRegressor
import xgboost as xgb

from mealpy.swarm_based import MRFO
from mealpy.utils.problem import Problem
from mealpy.utils.space import FloatVar
import time
from tqdm import tqdm

from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
from scipy.stats.mstats import winsorize

import shap
import itertools

In [4]:
data = pd.read_excel("Joyce Cleaned(Oxalic Mix Result).xlsx", 
                     sheet_name="Sheet1", 
                     usecols=["Potato Peel (g)", "Yam Peel (g)", "Banana peel (g)", "CONCENTRATION"], 
                     skiprows=2,     
                     nrows=17)       

In [6]:
data

,Potato Peel (g),Yam Peel (g),Banana peel (g),CONCENTRATION
0,2.50,10.00,2.50,16.000813
1,7.50,0.00,7.50,9.548173
2,15.00,0.00,0.00,8.943238
3,0.00,7.50,7.50,11.524294
4,6.25,2.50,6.25,9.427186
5,0.00,15.00,0.00,10.475740
6,2.50,2.50,10.00,10.637056
7,7.50,7.50,0.00,10.838701
8,0.00,7.50,7.50,8.418961
9,0.00,0.00,15.00,6.281524


In [8]:
# === 1. FEATURES & TARGET ===
X = data.drop(columns=['CONCENTRATION']).values
Y = data['CONCENTRATION'].values

In [10]:
SEED = 75
np.random.seed(SEED)

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=SEED)

In [14]:
best_score = -np.inf
best_model = None

def objective(trial): 
    global best_score, best_model


    params = {
    "n_estimators": trial.suggest_int("n_estimators", 100, 50000, step=5),
    "max_depth": trial.suggest_int("max_depth", 1, 5),
    "learning_rate": trial.suggest_float("learning_rate", 0.5, 1.0, log=True),  # Keep it moderate
    "subsample": trial.suggest_float("subsample", 0.9, 1.0),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.9, 1.0),
    "gamma": trial.suggest_float("gamma", 0.0, 0.3),
    "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 0.1),
    "reg_lambda": trial.suggest_float("reg_lambda", 1, 5.0),
    "min_child_weight": trial.suggest_int("min_child_weight", 3, 4),
    "max_delta_step": trial.suggest_int("max_delta_step", 0, 10),

}
    


    # Train the model
    model = XGBRegressor(
        **params,
        objective='reg:squarederror',
        random_state=SEED,
        n_jobs=-1,
        verbosity=0
    )

    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = cross_val_score(model, x_train, y_train, cv=kf, scoring="r2")
    mean_score = np.mean(scores)

    if mean_score > best_score:
        best_score = mean_score
        best_model = model
        model.fit(x_train, y_train)
    

    return mean_score

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

print("Best CV R²:", best_score)
print("Best Parameters:", study.best_params)

[I 2025-08-22 08:22:19,472] A new study created in memory with name: no-name-51b26f29-558f-42e9-bc2f-437df07607f9
[I 2025-08-22 08:22:48,014] Trial 0 finished with value: -4.178384804699731 and parameters: {'n_estimators': 10660, 'max_depth': 3, 'learning_rate': 0.5874584717601564, 'subsample': 0.9531849991259993, 'colsample_bytree': 0.9797930413534808, 'gamma': 0.25273257906958774, 'reg_alpha': 0.006207350556233593, 'reg_lambda': 3.175384878110131, 'min_child_weight': 4, 'max_delta_step': 4}. Best is trial 0 with value: -4.178384804699731.
[I 2025-08-22 08:23:13,830] Trial 1 finished with value: -5.105599512269166 and parameters: {'n_estimators': 12110, 'max_depth': 4, 'learning_rate': 0.5854864845719274, 'subsample': 0.9145128969995644, 'colsample_bytree': 0.9686617599424838, 'gamma': 0.26050664731811635, 'reg_alpha': 0.045626041393351, 'reg_lambda': 3.6151302978964, 'min_child_weight': 3, 'max_delta_step': 5}. Best is trial 0 with value: -4.178384804699731.
[I 2025-08-22 08:23:22,51

Best CV R²: -1.4004378305462453
Best Parameters: {'n_estimators': 26390, 'max_depth': 3, 'learning_rate': 0.5090602691584325, 'subsample': 0.9987827314942008, 'colsample_bytree': 0.9229109253587827, 'gamma': 0.2860692347664094, 'reg_alpha': 0.07289322008145702, 'reg_lambda': 4.443311935266859, 'min_child_weight': 3, 'max_delta_step': 0}


In [18]:
# Predict
y_train_pred = best_model.predict(x_train)
y_test_pred = best_model.predict(x_test)

# R² Scores
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print(f"Train R²: {train_r2:.4f}")
print(f"Test R²: {test_r2:.4f}")

Train R²: 0.8588
Test R²: 0.8051


In [45]:
# Display best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)
print("Best validation R² Score from Optuna:", study.best_value)

Best Hyperparameters: {'n_estimators': 26390, 'max_depth': 3, 'learning_rate': 0.5090602691584325, 'subsample': 0.9987827314942008, 'colsample_bytree': 0.9229109253587827, 'gamma': 0.2860692347664094, 'reg_alpha': 0.07289322008145702, 'reg_lambda': 4.443311935266859, 'min_child_weight': 3, 'max_delta_step': 0}
Best validation R² Score from Optuna: -1.4004378305462453


In [42]:
joblib.dump(best_model, "the_best_xgboost_oxalic_mix_model.pkl")

NameError: name 'best_model' is not defined

In [49]:
xgb_model = joblib.load('the_best_xgboost_oxalic_mix_model.pkl')
# Predict on the full dataset
y_pred_full = best_model.predict(X)

# Calculate R² score for the full dataset
r2_full = r2_score(Y, y_pred_full)
print(f'R² Score for the full dataset: {r2_full:.4f}')

R² Score for the full dataset: 0.8491


In [51]:
mse_full = mean_squared_error(Y, y_pred_full)
print(f'Mean Squared Error for the full dataset: {mse_full:.4f}')

Mean Squared Error for the full dataset: 0.7168


In [53]:
rmse_full = np.sqrt(mean_squared_error(Y, y_pred_full))
print(f'Root Mean Squared Error for the full dataset: {rmse_full:.4f}')

Root Mean Squared Error for the full dataset: 0.8466


In [55]:
# Calculate Average Absolute Deviation (AAD) for the full dataset
aad_full = np.mean(np.abs(Y.ravel() - y_pred_full))
print(f'Average Absolute Deviation (AAD) for the full dataset: {aad_full:.4f}')

Average Absolute Deviation (AAD) for the full dataset: 0.6802


In [57]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Make sure Y and y_pred_full are unscaled
n = len(Y)
rss = np.sum((Y.ravel() - y_pred_full) ** 2)

# Approximate parameter count
k = X.shape[1]  # or: k = best_model.get_params().get("n_estimators", X.shape[1])

# AIC formula
aic = n * np.log(rss / n) + 2 * k
print(f"AIC (approximate): {aic:.4f}")

AIC (approximate): 0.6719


In [ ]:
# Create the dataframe
data_full = pd.DataFrame(
    X,
    columns=['Potato Peel (g)', 'Yam Peel (g)', 'Banana peel (g)']
    )		
)
data_full['Actual Oxalic Mix Concntration'] = Y
data_full['Predicted Oxalic Mix Concentration'] = y_pred_full 

data_full

**MRFO OPTIMIZATION**

In [6]:
xgb_model = joblib.load('the_best_xgboost_oxalic_mix_model.pkl')

In [2]:
from mealpy.swarm_based import MRFO
import numpy as np
from mealpy.utils.problem import Problem
from mealpy.utils.space import FloatVar
import time
from tqdm import tqdm
import joblib

In [4]:
# ======================
# Load your trained model
# ======================
xgb_model = joblib.load('the_best_xgboost_oxalic_mix_model.pkl')


In [6]:
# ======================
# Define Search Space (unscaled, 0–15 each)
# ======================
search_space = [
    FloatVar(0, 15),  # Potato Peel
    FloatVar(0, 15),  # Yam Peel
    FloatVar(0, 15),  # Banana Peel
]

In [8]:
# ======================
# MRFO Hyperparameter Grid
# ======================
mrfo_hyperparam_space = {
    "epoch": range(5, 51, 5),
    "pop_size": range(10, 101, 10),
    "somersault_range": [1.0, 1.5, 2.0]
}

In [10]:
# ======================
# Projection function (no zero values)
# ======================
def project_to_constraint(solution, eps=0.01):
    """
    Projects any [A,B,C] to satisfy:
    - 0 < A,B,C <= 15 (no zero)
    - A+B+C = 15
    """
    sol = np.clip(solution, eps, 15)  # ensure minimum > 0
    total = np.sum(sol)
    if total == 0:
        # fallback: split evenly above eps
        return np.array([5.0, 5.0, 5.0])
    # rescale so that the sum = 15
    sol = sol * (15.0 / total)
    return np.clip(sol, eps, 15)

In [12]:
# ======================
# Objective Function
# ======================
def objective_function(solution):
    sol = project_to_constraint(np.array(solution))
    X_input = sol.reshape(1, -1)
    y_pred = xgb_model.predict(X_input)[0]

    if y_pred < 0 or y_pred > 100:
        return 1e6  # penalize nonsense
    return -y_pred  # we want to maximize concentration

In [14]:
# ======================
# Custom Problem
# ======================
class CustomProblem(Problem):
    def __init__(self):
        super().__init__(bounds=search_space, minmax="min", log_to=None)

    def obj_func(self, solution):
        return objective_function(solution)

In [16]:
# ======================
# MRFO Tuning
# ======================
def tune_mrfo_hyperparameters():
    best_removal, best_inputs, best_hyperparams = -np.inf, None, None
    total_runs = (len(mrfo_hyperparam_space["epoch"]) *
                  len(mrfo_hyperparam_space["pop_size"]) *
                  len(mrfo_hyperparam_space["somersault_range"]))

    print(f"\n Starting MRFO optimization over {total_runs} combinations...\n")
    start_time = time.time()

    with tqdm(total=total_runs, desc="Optimizing", ncols=100) as pbar:
        for epoch in mrfo_hyperparam_space["epoch"]:
            for pop_size in mrfo_hyperparam_space["pop_size"]:
                for somersault_range in mrfo_hyperparam_space["somersault_range"]:
                    run_start = time.time()
                    problem = CustomProblem()
                    optimizer = MRFO.OriginalMRFO(epoch=epoch, pop_size=pop_size,
                                                  somersault_range=somersault_range)
                    result = optimizer.solve(problem, seed=93)
                    run_end = time.time()

                    predicted_removal = -result.target.fitness
                    if predicted_removal > best_removal:
                        best_removal = predicted_removal
                        best_hyperparams = (epoch, pop_size, somersault_range)
                        best_inputs = project_to_constraint(result.solution)

                    pbar.set_postfix({
                        "Epoch": epoch, "Pop": pop_size, "Som": somersault_range,
                        "Best Conc": f"{best_removal:.2f}",
                        "Time(s)": f"{run_end - run_start:.1f}"
                    })
                    pbar.update(1)

    print(f"\n Optimization completed in {time.time() - start_time:.2f} seconds.")
    return best_hyperparams, best_inputs, best_removal

In [18]:
# ======================
# Run Optimization
# ======================
best_hyperparameters, best_inputs, best_removal = tune_mrfo_hyperparameters()


 Starting MRFO optimization over 300 combinations...



Optimizing: 100%|█| 300/300 [51:27<00:00, 10.29s/it, Epoch=50, Pop=100, Som=2, Best Conc=13.23, Time


 Optimization completed in 3087.13 seconds.


In [20]:
# ======================
# Show Results
# ======================
print("\n🔹 Best MRFO Hyperparameters:")
print(f"  - Epoch: {best_hyperparameters[0]}")
print(f"  - Population Size: {best_hyperparameters[1]}")
print(f"  - Somersault Range: {best_hyperparameters[2]}")

print("\n🔹 Optimized Input Values (Peels in g):")
print(f"  - Potato Peel:  {best_inputs[0]:.2f}")
print(f"  - Yam Peel:     {best_inputs[1]:.2f}")
print(f"  - Banana Peel:  {best_inputs[2]:.2f}")
print(f"  - Sum:          {np.sum(best_inputs):.2f}")

print(f"\n🔹 Best Optimized Concentration: {best_removal:.2f}")


🔹 Best MRFO Hyperparameters:
  - Epoch: 5
  - Population Size: 10
  - Somersault Range: 2.0

🔹 Optimized Input Values (Peels in g):
  - Potato Peel:  2.93
  - Yam Peel:     12.07
  - Banana Peel:  0.01
  - Sum:          15.00

🔹 Best Optimized Concentration: 13.23


In [22]:
# ======================
# Define Search Space (unscaled, 0–15 each)
# ======================
search_space = [
    FloatVar(0.5, 15),  # Potato Peel
    FloatVar(0.5, 15),  # Yam Peel
    FloatVar(0.5, 15),  # Banana Peel
]

In [24]:
# ======================
# MRFO Hyperparameter Grid
# ======================
mrfo_hyperparam_space = {
    "epoch": range(5, 51, 5),
    "pop_size": range(10, 101, 10),
    "somersault_range": [1.0, 1.5, 2.0]
}

In [26]:
# ======================
# Projection function (no zero values)
# ======================
def project_to_constraint(solution, eps=0.05):
    """
    Projects any [A,B,C] to satisfy:
    - 0 < A,B,C <= 15 (no zero)
    - A+B+C = 15
    """
    sol = np.clip(solution, eps, 15)  # ensure minimum > 0
    total = np.sum(sol)
    if total == 0:
        # fallback: split evenly above eps
        return np.array([5.0, 5.0, 5.0])
    # rescale so that the sum = 15
    sol = sol * (15.0 / total)
    return np.clip(sol, eps, 15)

In [28]:
# ======================
# Objective Function
# ======================
def objective_function(solution):
    sol = project_to_constraint(np.array(solution))
    X_input = sol.reshape(1, -1)
    y_pred = xgb_model.predict(X_input)[0]

    if y_pred < 0 or y_pred > 100:
        return 1e6  # penalize nonsense
    return -y_pred  # we want to maximize concentration

In [30]:
# ======================
# Custom Problem
# ======================
class CustomProblem(Problem):
    def __init__(self):
        super().__init__(bounds=search_space, minmax="min", log_to=None)

    def obj_func(self, solution):
        return objective_function(solution)

In [32]:
# ======================
# MRFO Tuning
# ======================
def tune_mrfo_hyperparameters():
    best_removal, best_inputs, best_hyperparams = -np.inf, None, None
    total_runs = (len(mrfo_hyperparam_space["epoch"]) *
                  len(mrfo_hyperparam_space["pop_size"]) *
                  len(mrfo_hyperparam_space["somersault_range"]))

    print(f"\n Starting MRFO optimization over {total_runs} combinations...\n")
    start_time = time.time()

    with tqdm(total=total_runs, desc="Optimizing", ncols=100) as pbar:
        for epoch in mrfo_hyperparam_space["epoch"]:
            for pop_size in mrfo_hyperparam_space["pop_size"]:
                for somersault_range in mrfo_hyperparam_space["somersault_range"]:
                    run_start = time.time()
                    problem = CustomProblem()
                    optimizer = MRFO.OriginalMRFO(epoch=epoch, pop_size=pop_size,
                                                  somersault_range=somersault_range)
                    result = optimizer.solve(problem, seed=93)
                    run_end = time.time()

                    predicted_removal = -result.target.fitness
                    if predicted_removal > best_removal:
                        best_removal = predicted_removal
                        best_hyperparams = (epoch, pop_size, somersault_range)
                        best_inputs = project_to_constraint(result.solution)

                    pbar.set_postfix({
                        "Epoch": epoch, "Pop": pop_size, "Som": somersault_range,
                        "Best Conc": f"{best_removal:.2f}",
                        "Time(s)": f"{run_end - run_start:.1f}"
                    })
                    pbar.update(1)

    print(f"\n Optimization completed in {time.time() - start_time:.2f} seconds.")
    return best_hyperparams, best_inputs, best_removal

In [34]:
# ======================
# Run Optimization
# ======================
best_hyperparameters, best_inputs, best_removal = tune_mrfo_hyperparameters()


 Starting MRFO optimization over 300 combinations...



Optimizing: 100%|█| 300/300 [1:02:57<00:00, 12.59s/it, Epoch=50, Pop=100, Som=2, Best Conc=13.23, Ti


 Optimization completed in 3777.64 seconds.


In [36]:
# ======================
# Show Results
# ======================
print("\n🔹 Best MRFO Hyperparameters:")
print(f"  - Epoch: {best_hyperparameters[0]}")
print(f"  - Population Size: {best_hyperparameters[1]}")
print(f"  - Somersault Range: {best_hyperparameters[2]}")

print("\n🔹 Optimized Input Values (Peels in g):")
print(f"  - Potato Peel:  {best_inputs[0]:.2f}")
print(f"  - Yam Peel:     {best_inputs[1]:.2f}")
print(f"  - Banana Peel:  {best_inputs[2]:.2f}")
print(f"  - Sum:          {np.sum(best_inputs):.2f}")

print(f"\n🔹 Best Optimized Concentration: {best_removal:.2f}")


🔹 Best MRFO Hyperparameters:
  - Epoch: 5
  - Population Size: 10
  - Somersault Range: 2.0

🔹 Optimized Input Values (Peels in g):
  - Potato Peel:  3.23
  - Yam Peel:     11.39
  - Banana Peel:  0.38
  - Sum:          15.00

🔹 Best Optimized Concentration: 13.23


In [48]:
# === 1. IMPORTS ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import joblib
import optuna

# Scikit-learn modules
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.inspection import PartialDependenceDisplay
from sklearn.inspection import partial_dependence
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error
from sklearn.base import BaseEstimator, RegressorMixin

# XGBoost Regressor
from xgboost import XGBRegressor
import xgboost as xgb

from mealpy.swarm_based import MRFO
from mealpy.utils.problem import Problem
from mealpy.utils.space import FloatVar
import time
from tqdm import tqdm

from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
from scipy.stats.mstats import winsorize

import shap
import itertools

In [50]:
xgb_model = joblib.load('the_best_xgboost_oxalic_mix_model.pkl')

In [52]:
# Example: load Sheet1, only rows 2–11 (Excel rows, meaning index 1–10 in Python)
data = pd.read_excel("Joyce Cleaned(Oxalic Mix Result).xlsx", 
                     sheet_name="Sheet1", 
                     usecols=["Potato Peel (g)", "Yam Peel (g)", "Banana peel (g)", "CONCENTRATION"], 
                     skiprows=2,    # skip the first row
                     nrows=17)      # read the next 10 rows#

In [54]:
data

,Potato Peel (g),Yam Peel (g),Banana peel (g),CONCENTRATION
0,2.50,10.00,2.50,16.000813
1,7.50,0.00,7.50,9.548173
2,15.00,0.00,0.00,8.943238
3,0.00,7.50,7.50,11.524294
4,6.25,2.50,6.25,9.427186
5,0.00,15.00,0.00,10.475740
6,2.50,2.50,10.00,10.637056
7,7.50,7.50,0.00,10.838701
8,0.00,7.50,7.50,8.418961
9,0.00,0.00,15.00,6.281524


In [56]:
# === 1. FEATURES & TARGET ===
X = data.drop(columns=['CONCENTRATION']).values
Y = data['CONCENTRATION'].values

In [58]:
# Predict on the full dataset
y_pred_full = xgb_model.predict(X)

# Calculate R² score for the full dataset
r2_full = r2_score(Y, y_pred_full)
print(f'R² Score for the full dataset: {r2_full:.4f}')

R² Score for the full dataset: 0.8491


In [60]:
mse_full = mean_squared_error(Y, y_pred_full)
print(f'Mean Squared Error for the full dataset: {mse_full:.4f}')

Mean Squared Error for the full dataset: 0.7168


In [62]:
rmse_full = np.sqrt(mean_squared_error(Y, y_pred_full))
print(f'Root Mean Squared Error for the full dataset: {rmse_full:.4f}')

Root Mean Squared Error for the full dataset: 0.8466


In [68]:
# Calculate Average Absolute Deviation (AAD) for the full dataset
aad_full = np.mean(np.abs(Y - y_pred_full))
print(f'Average Absolute Deviation (AAD) for the full dataset: {aad_full:.4f}')

Average Absolute Deviation (AAD) for the full dataset: 0.6802


In [72]:
# Calculate AIC for the full dataset
n = len(Y)
residual_sum_of_squares = np.sum((Y - y_pred_full) ** 2)
k = len(xgb_model.get_params())  # number of model parameters

aic_full = n * np.log(residual_sum_of_squares / n) + 2 * k
print(f'AIC for the full dataset: {aic_full:.4f}')

AIC for the full dataset: 74.6719


In [76]:
# Create the dataframe
data_full = pd.DataFrame(
    X,
    columns=['Potato Peel (g)', 'Yam Peel (g)', 'Banana peel (g)']
    )		

data_full['Actual Oxalic Mix Concntration'] = Y
data_full['Predicted Oxalic Mix Concentration'] = y_pred_full 

data_full

,Potato Peel (g),Yam Peel (g),Banana peel (g),Actual Oxalic Mix Concntration,Predicted Oxalic Mix Concentration
0,2.50,10.00,2.50,16.000813,15.214657
1,7.50,0.00,7.50,9.548173,9.570955
2,15.00,0.00,0.00,8.943238,8.409145
3,0.00,7.50,7.50,11.524294,9.374676
4,6.25,2.50,6.25,9.427186,10.721840
5,0.00,15.00,0.00,10.475740,10.706541
6,2.50,2.50,10.00,10.637056,10.721840
7,7.50,7.50,0.00,10.838701,10.063129
8,0.00,7.50,7.50,8.418961,9.374676
9,0.00,0.00,15.00,6.281524,6.896242
